## Libraries

In [15]:
import matplotlib
matplotlib.use('TkAgg')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from new_datasets_py import create_subsets

## Load file

In [6]:
# Load the data
data = pd.read_csv('crypto-markets.csv')
btc_data = data[data['symbol'] == 'BTC'].copy()
doge_data = data[data['symbol'] == 'DOGE'].copy()
ltc_data = data[data['symbol'] == 'LTC'].copy()
btc_data.loc[:, 'date'] = pd.to_datetime(btc_data['date'])
doge_data.loc[:, 'date'] = pd.to_datetime(doge_data['date'])
ltc_data.loc[:, 'date'] = pd.to_datetime(ltc_data['date'])
btc_data.set_index('date', inplace=True)
doge_data.set_index('date', inplace=True)
ltc_data.set_index('date', inplace=True)
#missing_values = btc_data.isnull().sum() + doge_data.isnull().sum() + ltc_data.isnull().sum()
#print("Missing values:\n", missing_values)
cr_subsets = create_subsets()

/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/opt/homebrew/lib/python3.11/site-packages/pandas/core/indexes/base.py:7587: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(seq

## Creating subsets

In [3]:
btc_subsets = cr_subsets.create_random_subsets(btc_data)
doge_subsets = cr_subsets.create_random_subsets(doge_data)
ltc_subsets = cr_subsets.create_random_subsets(ltc_data)

In [4]:
btc_table = cr_subsets.create_dataset_table(btc_subsets)
doge_table = cr_subsets.create_dataset_table(doge_subsets)
ltc_table = cr_subsets.create_dataset_table(ltc_subsets)

In [7]:
btc_table_with_label = cr_subsets.add_label_column(btc_table)
ltc_table_with_label = cr_subsets.add_label_column(ltc_table)
doge_table_with_label = cr_subsets.add_label_column(doge_table)

print(doge_table_with_label)

    dataset_id                                              dates  \
0            1  [2018-01-27, 2018-01-28, 2018-01-29, 2018-01-3...   
1            2  [2016-08-07, 2016-08-08, 2016-08-09, 2016-08-1...   
2            3  [2017-09-12, 2017-09-13, 2017-09-14, 2017-09-1...   
3            4  [2017-06-24, 2017-06-25, 2017-06-26, 2017-06-2...   
4            5  [2018-01-13, 2018-01-14, 2018-01-15, 2018-01-1...   
5            6  [2014-02-11, 2014-02-12, 2014-02-13, 2014-02-1...   
6            7  [2016-12-15, 2016-12-16, 2016-12-17, 2016-12-1...   
7            8  [2015-09-14, 2015-09-15, 2015-09-16, 2015-09-1...   
8            9  [2017-12-20, 2017-12-21, 2017-12-22, 2017-12-2...   
9           10  [2017-05-14, 2017-05-15, 2017-05-16, 2017-05-1...   
10          11  [2017-11-16, 2017-11-17, 2017-11-18, 2017-11-1...   
11          12  [2016-03-07, 2016-03-08, 2016-03-09, 2016-03-1...   
12          13  [2014-04-01, 2014-04-02, 2014-04-03, 2014-04-0...   
13          14  [2014-02-21, 2014-

## Arima init

In [24]:
from statsmodels.tsa.arima.model import ARIMA

rise_result = []
combined_table = pd.concat([doge_table_with_label, ltc_table_with_label, btc_table_with_label])

TP, TN, FP, FN = 0, 0, 0, 0

for index, row in combined_table.iterrows():
    training_data = row['close_values'][:7]
    model = ARIMA(training_data, order=(2,1,1))
    model_fit = model.fit()

    forecast_test = model_fit.forecast(3)

    predicted_rise = 1 if forecast_test[2] > row['close_values'][6] else 0
    actual_rise = row['rise']

    if predicted_rise == 1 and actual_rise == 1:
        TP += 1
    elif predicted_rise == 0 and actual_rise == 0:
        TN += 1
    elif predicted_rise == 1 and actual_rise == 0:
        FP += 1
    elif predicted_rise == 0 and actual_rise == 1:
        FN += 1

print("True Positives (TP):", TP)
print("True Negatives (TN):", TN)
print("False Positives (FP):", FP)
print("False Negatives (FN):", FN)

# btc_data_daily['close'].plot(label='Original Data')
# print(forecast_test)

# data[data['symbol'] == 'BTC'].copy()
# model = ARIMA(training_data, order=(4,2,1))
# model_fit = model.fit()
# print(model_fit.summary())

/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximu

True Positives (TP): 21
True Negatives (TN): 48
False Positives (FP): 29
False Negatives (FN): 22


/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
sensitivity = TP / (TP + FN)
specificity = TN / (FP + TN)
precision = TP / (TP + FP)
accuracy = (TP + TN) / (TP + TN + FP + FN)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Precision", precision)
print("Accuracy", accuracy)